In [79]:
from selenium import webdriver # Importa driver para navegadores pelo selenium
from selenium.webdriver.common.by import By # Importando classe By que server para selecionar os elementos
from selenium.webdriver.common.keys import Keys # Importo todas as teclas do teclado pelo pacote do Selenium
from selenium.webdriver.support.ui import Select # Import necessario para selecao de elementos em dropdown/selector
from selenium.webdriver.support.wait import WebDriverWait # Import da classe WAIT, serve para dizer o quanto se deve esperar pro navegador
from selenium.webdriver.support import expected_conditions as EC # Import de expected_conditions, e da nome reduzido (alias) EC

driver = webdriver.Chrome() # Declara que driver é do chrome

In [80]:
from dataclasses import dataclass

@dataclass
class Usuario:
    foto: str
    nome: str
    profissao: str
    email: str
    telefone: str
    user: str
    estado: str


In [81]:
driver.get('http://localhost:8000/#/desafio/2') # Declaro pagina que estou acessando

# Usuarios a serem buscados

- Dalton
- Emily
- Nadia
- Joao
- Leonardo
- Silvana
- Rafael

In [82]:
listaPesquisa = ['Dalton', 'Emily', 'Nadia', 'Joao', 'Leonardo', 'Silvana', 'Rafael']

In [83]:
driver.implicitly_wait(time_to_wait=10)

In [87]:
wait = WebDriverWait(driver=driver, timeout=15, poll_frequency=1) 


In [86]:
pesquisa = driver.find_element(By.CSS_SELECTOR, 'main input[type="text"]')
pesquisaBtn = driver.find_element(By.CSS_SELECTOR, 'main button')

data = []

In [88]:
for pessoa in listaPesquisa:
    
    # Preenche info de pesquisa
    pesquisa.clear()
    pesquisa.send_keys(pessoa)

    # Espero ate que botao seja clickavel para pesquisar e fazer a pesquisa
    wait.until(EC.element_to_be_clickable(mark=pesquisaBtn))
    pesquisaBtn.click()

    # Obtenho todos usuarios na busca (Espero imagens serem carregadas)
    locator = (By.CSS_SELECTOR, 'div.users-list > div > img')
    wait.until(EC.visibility_of_all_elements_located(locator=locator))
    usuarios = driver.find_elements(By.CSS_SELECTOR, 'div.users-list > div')

    for u in usuarios:

        imgPath = u.find_element(By.TAG_NAME, 'img').get_attribute('src')
        nome = u.find_element(By.TAG_NAME, 'h3').text
        profissao = u.find_element(By.TAG_NAME, 'span').text
        email = u.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(1)').text
        telefone = u.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(2)').text
        user = u.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(3)').text
        estado = u.find_element(By.CSS_SELECTOR, 'ul > li:nth-child(4)').text

        userData = Usuario(
            foto = imgPath,
            nome = nome,
            profissao = profissao,
            email = email[8:],
            telefone = telefone[10:],
            user = user[9:],
            estado = estado[8:]
        )

        data.append(userData)

# Salvando em JSON

In [94]:
from dataclasses import asdict # Converte instancia do dataclass pra dicionario do python
import json

In [97]:
asdict(data[0])

{'foto': 'https://avatars.githubusercontent.com/u/22959227',
 'nome': 'Dalton Martins',
 'profissao': 'Gerente',
 'email': 'dalton.martins22@example.org',
 'telefone': '(84) 23117-2586',
 'user': 'dalton.martins44',
 'estado': 'Paraíba'}

In [102]:
with open('dadosCapturados.json', 'w') as file:
    dataFormatted = [asdict(d) for d in data]
    json.dump(dataFormatted, file, ensure_ascii=True)

In [103]:
import csv
from dataclasses import fields

In [105]:
with open('dadosCapturados.csv', 'w') as csvFile:
    headers = [field.name for field in fields(Usuario)]
    file = csv.DictWriter(csvFile, fieldnames=headers)
    
    file.writeheader()
    dataFormatted = [asdict(d) for d in data]
    file.writerows(dataFormatted)